In [1]:
import os
import pandas as pd
from sklearn.ensemble import GradientBoostingRegressor
import shap

Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)


# Preparation of dataset.

In [2]:
def load_excel(path):
    df = pd.read_excel(path)
    return df

In [3]:
def process_dataset(df):
    # Valid data with sd<0.1.
    df = df[df["K_ave_sd"]<0.1]
    
    # Rename columns.
    col_rename = {}
    for col in df.columns[5:-2]:
        col_new = col.replace("log10[","").replace("]\n(mM)","")
        col_rename[col] = col_new
    col_rename["Strain (gene)"] = "Gene"
    df = df.rename(columns = col_rename)
    
    # Remove columns of H (Not considered in this study).
    df = df.drop("H",axis=1)

    return df

In [4]:
# Import data.
raw_data = load_excel(os.getcwd()+"/Data.xlsx")

# Process data.
data = process_dataset(raw_data)

In [5]:
data

,Unique ID,Combination ID,Medium ID,Condition ID_unique,Gene,Glucose,Citrate,Ammonium,Phosphate,Sulfate,...,VB3,VB5,VB6,VB8,VB9,VB12,Borate,PABA,K_ave,K_ave_sd
0,C00L2-1_btuR,C00L2_btuR,C00L2,C00L2-1,btuR,0.376577,-2.744727,0.514548,1.820595,0.209431,...,-3.267606,-3.899629,-4.829856,-5.899629,-6.443697,-7.744727,-3.142668,-3.267606,0.11000,0.018129
1,C00L2-1_cobB,C00L2_cobB,C00L2,C00L2-1,cobB,0.376577,-2.744727,0.514548,1.820595,0.209431,...,-3.267606,-3.899629,-4.829856,-5.899629,-6.443697,-7.744727,-3.142668,-3.267606,0.11850,0.017078
2,C00L2-1_mazG,C00L2_mazG,C00L2,C00L2-1,mazG,0.376577,-2.744727,0.514548,1.820595,0.209431,...,-3.267606,-3.899629,-4.829856,-5.899629,-6.443697,-7.744727,-3.142668,-3.267606,0.10750,0.000707
3,C00L2-1_mog,C00L2_mog,C00L2,C00L2-1,mog,0.376577,-2.744727,0.514548,1.820595,0.209431,...,-3.267606,-3.899629,-4.829856,-5.899629,-6.443697,-7.744727,-3.142668,-3.267606,0.14500,0.001414
4,C00L2-1_ilvH,C00L2_ilvH,C00L2,C00L2-1,ilvH,0.376577,-2.744727,0.514548,1.820595,0.209431,...,-3.267606,-3.899629,-4.829856,-5.899629,-6.443697,-7.744727,-3.142668,-3.267606,0.11975,0.007632
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15475,C0138-1_pdxY,C0138_pdxY,C0138,C0138-1,pdxY,2.260548,-2.744727,1.968483,2.202856,1.022988,...,-3.267606,-5.899629,-6.744727,-7.899629,-3.443697,-6.045757,-1.443697,-0.267606,0.28450,0.024749
15483,C0138-1_thiF,C0138_thiF,C0138,C0138-1,thiF,2.260548,-2.744727,1.968483,2.202856,1.022988,...,-3.267606,-5.899629,-6.744727,-7.899629,-3.443697,-6.045757,-1.443697,-0.267606,0.32100,0.038184
15494,C0138-1_acpT,C0138_acpT,C0138,C0138-1,acpT,2.260548,-2.744727,1.968483,2.202856,1.022988,...,-3.267606,-5.899629,-6.744727,-7.899629,-3.443697,-6.045757,-1.443697,-0.267606,0.19150,0.036062
15495,C0138-1_gabD,C0138_gabD,C0138,C0138-1,gabD,2.260548,-2.744727,1.968483,2.202856,1.022988,...,-3.267606,-5.899629,-6.744727,-7.899629,-3.443697,-6.045757,-1.443697,-0.267606,0.33800,0.036770


# Perform GBDT for the all dataset.

In [6]:
def load_csv(path):
    df = pd.read_csv(path, index_col=0)
    return df

In [7]:
# Perform GBDT and calculate feature importances.
def GBDT_strain(df, hyperparams, variables, param):
    list_ip_series=[]
    strain_counts={}

    # Trasform hyperparams type.
    hyperparams = trans(hyperparams)

    for strain in sorted(list(set(df["Gene"]))):
        strains = df[df["Gene"]==strain].dropna(subset=[param])
        hyperparams_strain = hyperparams[strain]

        if len(strains)!=0:
            df_in = strains[variables]
            df_out = strains[param]

            # Perform GBDT.
            cfr = GradientBoostingRegressor(**hyperparams_strain)
            clf = cfr.fit(df_in, df_out)

            #Calculate feature importances.
            ip_series = pd.Series({n:s for n, s in zip(variables, clf.feature_importances_)},name=strain)
            list_ip_series.append(ip_series)
            
    feature_importances = pd.concat(list_ip_series,axis=1)

    return feature_importances

#Transform types.
def trans(hyperparams):
    trans_hyperparams = {gene:{
        "learning_rate":hyperparams.at["learning_rate",gene],
        "max_depth":int(hyperparams.at["max_depth",gene]),
        "n_estimators":int(hyperparams.at["n_estimators",gene]),
        "random_state":int(hyperparams.at["random_state",gene])
        } for gene in hyperparams.columns}

    return trans_hyperparams

In [8]:
# Import hyperparams.
hyperparams = load_csv(os.getcwd()+"/hyperparams/best_hyperparams_all.csv")

# Set variables and parameter.
variables = data.columns[5:-2]
param = "K_ave"

In [ ]:
# Perform GBDT.
feature_importances = GBDT_strain(data, hyperparams, variables, param)

In [ ]:
feature_importances

# Perform GBDT and SHAP for the K_sd dataset.

In [ ]:
def make_unique_data(df):
    # Extract unique conditions.
    df_unique = df[~df["Condition ID_unique"].duplicated()].set_index("Condition ID_unique").loc[:,"Glucose":"PABA"]
    
    # Calculate standard deviation for each condition.
    df_unique["K_std_cond"] = df.groupby("Condition ID_unique")["K_ave"].std() 

    return df_unique

In [ ]:
# Perform SHAP.
def shap_analysis(df, hyperparams, variables, param):
    df_in = df[variables]
    df_out = df[param]
    
    def GBDT(df_in, df_out, hyperparams, variables, param):
        hyperparams = trans(hyperparams)["K_std"]
    
        cfr = GradientBoostingRegressor(**hyperparams)
        clf = cfr.fit(df_in, df_out)
    
        return clf

    # Perform GBDT.
    clf = GBDT(df_in, df_out, hyperparams, variables, param)

    # Perform SHAP.
    shap.initjs()
    explainer = shap.Explainer(clf,df_in)
    shap_values = explainer(df_in)

    
    return shap_values.values

In [ ]:
# Make unique data.
unique_data = make_unique_data(data)

In [ ]:
unique_data

In [ ]:
# Import hyperparams.
hyperparams_Kstd = load_csv(os.getcwd()+"/hyperparams/best_hyperparams_Kstd.csv")

# Set variables and parameter.
variables = data.columns[5:-2]
param = "K_std_cond"

In [ ]:
shap_analysis(unique_data, hyperparams_Kstd, variables, param)

# Perform GBDT for the data sets divided by glucose concentration.

In [ ]:
def divide_dataset(df):
    #Divide the dataset by glucose concentration.
    df_glcL = df[df["Glucose"]<0.7]
    df_glcM = df[(df["Glucose"]>=0.7)&(df["Glucose"]<1.7)]
    df_glcH = df[df["Glucose"]>=1.7]

    return df_glcL, df_glcM, df_glcH

In [ ]:
# Make divided datasets.
data_glcL, data_glcM, data_glcH = divide_dataset(data)

# Import hyperparams.
hyperparams_glcL = load_csv(os.getcwd()+"/hyperparams/best_hyperparams_glcL.csv")
hyperparams_glcM = load_csv(os.getcwd()+"/hyperparams/best_hyperparams_glcM.csv")
hyperparams_glcH = load_csv(os.getcwd()+"/hyperparams/best_hyperparams_glcH.csv")

# Set variables and parameter.
variables = data.columns[5:-2]
param = "K_ave"

In [ ]:
# Perform GBDT.
feature_importances_glcL = GBDT_strain(data_glcL, hyperparams_glcL, variables, param)
feature_importances_glcM = GBDT_strain(data_glcM, hyperparams_glcM, variables, param)
feature_importances_glcH = GBDT_strain(data_glcH, hyperparams_glcH, variables, param)

In [ ]:
feature_importances_glcL

In [ ]:
feature_importances_glcM

In [ ]:
feature_importances_glcH